In [1]:
import torch
import random
from seeker import White_seeker, Random_select_seeker, Random_gen_seeker, Black_seeker
from utils import Unfair_metric, load_model, get_L_matrix
from data import adult
from train_dnn import get_data
from dnn_models.model import MLP, NormMLP
from distances.normalized_mahalanobis_distances import SquaredEuclideanDistance, ProtectedSEDistances
from distances.sensitive_subspace_distances import LogisticRegSensitiveSubspace
from distances.binary_distances import BinaryDistance
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from IPython.display import display

%load_ext autoreload
%autoreload 2

In [2]:
rand_seed = 0
use_protected_attr = True
protected_vars = ['sex_Male']
# protected_vars = ['sex_Male', 'race_White']

dataset, train_dl, test_dl = get_data(adult, rand_seed, protected_vars=protected_vars)
dataset.use_protected_attr = use_protected_attr
in_dim = dataset.dim_feature()
out_dim = 2

# prepare data
all_X, all_y = dataset.get_all_data(), dataset.labels

adult_gen = adult.Adult_gen(sensitive_columns=dataset.protected_idxs, include_protected_feature=use_protected_attr)

In [3]:
model = MLP(in_dim, out_dim)
load_model(model, 'MLP', 'adult', 'STDTrainer', use_protected_attr=use_protected_attr, \
           protected_vars=protected_vars, id=rand_seed)

# model = NormMLP(in_dim, out_dim, adult_gen.get_range('data'))
# load_model(model, 'NormMLP', 'adult', 'STDTrainer', use_protected_attr=use_protected_attr, \
#            protected_vars=protected_vars, id=rand_seed)

all_pred = model.get_prediction(all_X)

In [4]:
# prepare distances
distance_x_NSE = SquaredEuclideanDistance()
distance_x_Causal = ProtectedSEDistances()
distance_x_LR = LogisticRegSensitiveSubspace()
distance_y = BinaryDistance()

distance_x_NSE.fit(num_dims=dataset.dim_feature(), data_gen=adult_gen)
distance_x_Causal.fit(num_dims=dataset.dim_feature(), data_gen=adult_gen, protected_idx=dataset.protected_idxs)
distance_x_LR.fit(all_X, adult_gen, protected_idxs=dataset.protected_idxs)

In [5]:
def rand_gen():
    return {
        'age': random.randint(15, 60),
        'capital_gain': 0,
        'capital_loss': 0,
        'education_num': random.randint(1, 15),
        'hours_per_week': random.randint(10, 50),
        'race_white': random.choice([0, 1]),
        'sex_male': random.choice([0, 1]),
        'marital_status': random.choice(list(range(7))),
        'occupation': random.choice(list(range(14))),
        'relationship': random.choice(list(range(6))),
        'workclass': random.choice(list(range(7)))
    }

def perturb_pair(x, pert_features, pert_func):
    pair = dict()
    for k, v in x.items():
        pair[k] = torch.tensor([v, v])
        if k in pert_features:
            pair[k][1] = pert_func(pair[k][0])
    return pair

x = rand_gen()
# pair = adult.generate_from_origin(**perturb_pair(x, ['sex_male', 'race_white'], lambda x:1-x))
# adult.get_original_feature(pair)

In [6]:
pair = adult.generate_from_origin(**perturb_pair(x, ['capital_loss'], lambda x:1+x))
adult.get_original_feature(pair)

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,38.0,0.0,0.0,12.0,27.0,0.0,1.0,4.0,11.0,0.0,1.0
1,38.0,0.0,1.0,12.0,27.0,0.0,1.0,4.0,11.0,0.0,1.0


In [7]:
# dist = distance_x_LR(all_X[0], all_X[0] + torch.eye(all_X.shape[1]), itemwise_dist=False).squeeze()
# print(dist)
# print(dist[dataset.protected_idxs])

In [8]:
# 1/1e-10

In [9]:
dist = distance_x_Causal(all_X[0], all_X[0] + torch.eye(all_X.shape[1]), itemwise_dist=False).squeeze()
print(dist)
print(dist[dataset.protected_idxs])

tensor([1.8765e-04, 1.0001e-10, 5.2702e-08, 4.4444e-03, 1.0412e-04, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])
tensor([0.])


In [10]:
# x = rand_gen()
# for key in x:
#     pair = adult.generate_from_origin(**perturb_pair(x, key, x[key]+1))
#     print(distance_x_NSE(pair[0], pair[1]), 1 / distance_x_NSE(pair[0], pair[1]))

In [11]:
# epsilon越大，要求不公平样本对dx越小，越严格
# epsilon = 1e10
epsilon = 9e9
unfair_metric = Unfair_metric(dx=distance_x_Causal, dy=distance_y, epsilon=epsilon)

In [12]:
L = get_L_matrix(all_X, all_pred, distance_x_Causal, distance_y)
n_unfair = torch.sum(L>epsilon).item()

total_pairs = all_X.shape[0]**2

unfair_ratio = n_unfair/total_pairs
unfair_ratio

  0%|          | 0/16 [00:00<?, ?it/s]

0.0

In [13]:
# 1/unfair_ratio

In [14]:
# p_success = lambda n: 1 - (1 - unfair_ratio)**n
# for i in range(10):
#     if p_success(10**i) > 1/100:
#         print(10**i, p_success(10**i))

In [15]:
def show_result(result):
    pair, n_query = result
    if pair != None:
        display(adult.get_original_feature(pair), n_query)
    else:
        print('not found')

In [16]:
# random.seed(422)
# torch.manual_seed(422)

# select_seeker = Random_select_seeker(model=model, unfair_metric=unfair_metric, data=all_X)
# for _ in range(5):
#     show_result(select_seeker.seek(dx_constraint=True, max_query=1e6))

In [17]:
# random.seed(422)
# torch.manual_seed(422)

# gen_seeker = Random_gen_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# for _ in range(3):
#     show_result(gen_seeker.seek(by_range=True, max_query=1e6))

In [18]:
# random.seed(422)
# torch.manual_seed(422)

# for _ in range(3):
#     show_result(gen_seeker.seek(by_range=False, max_query=1e6))

In [19]:
adult_gen.get_range('data')

tensor([[1.7000e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.0000e+01, 9.9999e+04, 4.3560e+03, 1.6000e+01, 9.9000e+01, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000

In [20]:
random.seed(422)
torch.manual_seed(422)

white_seeker = White_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# show_result(white_seeker.seek())
for _ in range(5):
    show_result(white_seeker.seek(max_query=1e6))

out1: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


41

out1: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


...

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


44

gradiant:
 tensor([[-0.0056,  0.0002,  0.0028, -0.0028, -0.0066, -0.0310,  0.0055,  0.0149,
         -0.0090, -0.0430, -0.0191, -0.0170, -0.0239,  0.0012, -0.0117,  0.0145,
         -0.0171, -0.0226, -0.0144, -0.0307, -0.0118,  0.0101,  0.0013, -0.0036,
          0.0033, -0.0122, -0.0097,  0.0163, -0.0392, -0.0183, -0.0325, -0.0318,
          0.0033, -0.0035, -0.0007, -0.0031, -0.0188,  0.0103, -0.0140, -0.0053,
         -0.0074]])
data scale:
 tensor([7.3000e+01, 9.9999e+04, 4.3560e+03, 1.5000e+01, 9.8000e+01, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])
te

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


61

In [21]:
random.seed(422)
torch.manual_seed(422)

black_seeker = Black_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
for _ in range(5):
    show_result(black_seeker.seek(max_query=1e6))

calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
out1: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


1335

calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
out1: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


...

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


1502

calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
x0 converge
tensor([], dtype=torch.int64)
(tensor([], dtype=torch.int64),)
restart
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
out1: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


2256